### Recommender System

&nbsp;

Recommender system is a system widely used in Amazon, Netflix to predict user rating for a given item. Usually the system involves collaborative filtering, content-based filtering, session-based filtering or even a mixture.

* Collaborative filtering seeks connections across different users and items to predict the rating. It is more related to unsupervised learning. Even inside collaborative filtering, there are a few different techniques such as model-based (matrix factorization, latent variables), memory-based (KNN, KNN with z score) and even a hybrid of both. Collaborative filtering will be the main focus of this script. 
* Content-based filtering collects user and item profile. Based upon the features in the profile, it forecasts the user preference towards different items. It is more related to supervised learning.
* Session-based filtering monitors the user interaction within a session to create recommendations. It is quite helpful to navigate through the cold start problem where a new user has not much available information for modelling. It is more related to deep learning, in particular, Recurrent Neural Network.

In a recommender system, not every customer rates every item so it effectively forms a partially filled customer vs item matrix. To recommend anything to the existing customer, a fully filled matrix must be inferred from the dataset. This falls into the spectrum of matrix completion. The most popular sub-problem of matrix completion is to find a low rank matrix via convex optimization (Candès and Recht, 2008). The sub-problem assumes there must be latent variables influencing the users and the items in the matrix. Thus, the matrix must be low rank.

Reference to image recovery style matrix completion

https://github.com/je-suis-tm/machine-learning/blob/master/matrix%20completion.ipynb

Reference to surprise library for recommender system (this script draws a lot of inspirations from it though much faster convergence due to the intensive usage of linear algebra rather than iterations)

https://surprise.readthedocs.io/en/stable/

In [1]:
import os
os.chdir('K:/ecole/github/televerser/matrix completion')
import pandas as pd
import numpy as np
import numba

### Read Files

In [2]:
#data comes from the link below
# http://files.grouplens.org/datasets/movielens/ml-100k/u.data
# https://github.com/je-suis-tm/machine-learning/blob/master/data/movielens.csv
df=pd.read_csv('movielens.csv')

In [3]:
#shrink data size for better performance
df=df[df['user']<=100][df['item']<=500].copy()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [4]:
#convert array into matrix form
matrix=df.pivot(index='item',columns='user',values='rating')
arr=np.array(matrix)

In [5]:
#filled elements
known_values=list(zip(np.where(~np.isnan(arr))[0],
        np.where(~np.isnan(arr))[1]))

#randomly select 30% as testing set
lucky_draw=set(np.random.choice(range(len(known_values)),
                 size=int(len(known_values)*0.3),
                 p=[1/len(known_values)]*len(known_values)))

unlucky_draw=[i for i in range(len(known_values)) if i not in lucky_draw]
testing_idx=(np.where(~np.isnan(arr))[0][list(lucky_draw)],
np.where(~np.isnan(arr))[1][list(lucky_draw)])
training_idx=(np.where(~np.isnan(arr))[0][list(unlucky_draw)],
np.where(~np.isnan(arr))[1][list(unlucky_draw)])

#train test split
mask=np.ones(arr.shape)
mask[testing_idx]=np.nan
arr_train=np.multiply(arr,mask)

### Model-based

#### Funk SVD

&nbsp;

Simon Funk developed a SVD-like latent factor model to win 3rd prize in 2006 Netflix problem. Conventionally people call it Funk SVD, although it is merely inspired by Singular Value Decomposition without explicitly using SVD. Funk SVD is really easy to implement and quick to converge with high accuracy. It is a type of collaborative filtering focusing on matrix factorization. The official optimization problem is formulated as below.

$$ \min_{p_*,q_*,b_*}\,\sum_{r_{ui}\,\in\,\mathcal{K}} \left(r_{ui} - \hat{r}_{ui} \right)^2 + \lambda( ||p_u||^2 + ||q_i||^2 + b_u^2 + b_i^2 )$$

where 

${r}_{ui}$ denotes the rating of item $i$ by user $u$

$\hat{r}_{ui}$ denotes the estimated rating of item $i$ by user $u$, it can be decomposed into the form of $\mu + b_u + b_i + q_i^Tp_u$

$\mathcal{K}$ denotes the observed user vs item matrix (partially filled)

$\mu$ denotes overall average rating

$b_u$ denotes the deviation from overall average rating caused by user $u$, can be seen as user baseline where some users are more critical so they give lower ratings in general

$b_i$ denotes the deviation from overall average rating caused by item $i$, can be seen as item baseline where some items are more appealing so they attract higher ratings in general

$p_u$ denotes the latent factors of user $u$, in translation, user preference

$q_i$ denotes the latent factors of item $i$, in translation, item attributes

$\lambda$ denotes LaGrange multiplier which is the coefficient of L2 penalty function

&nbsp;

To explain a bit, Funk SVD assumes that both user and item are affected by $m$ number of latent factors ($p_u \in \mathbb{R}^{m}\,\&\,q_i \in \mathbb{R}^{m}$). These latent factors could be percentage of action in the movie, number of tier 1 hollywood stars, etc. The rating $r_{ui}$ is merely the linear combination $q_i^Tp_u$ of user preference and item attributes. This decomposition is similar to SVD in the form of $U\Sigma V^T$ without the eigenvalue diagonal $\Sigma$ as scaling factors. Hence, the first bit of the objective function is an ordinary least square to minimize the sum of squared error between existing rating and estimated rating. Since the matrix is partially filled, the second bit of the objective function is L2 norm regularization on user preference $p_u$, item attributes $q_i$, user deviation $b_u$ and item deviation $b_i$ to avoid overfit problem.

The actual optimization solver is inspired by gradient descent $\theta:=\theta-\alpha\frac{\partial J(\theta)}{\partial \theta}$. Applying the same logic to every unknown parameters $p_u$, $q_i$, $b_u$ and $b_i$, we end up with the following updates.

$$b_u := b_u + \alpha (\epsilon_{ui} - \lambda b_u)$$
$$b_i := b_i + \alpha (\epsilon_{ui} - \lambda b_i)$$
$$p_u := p_u + \alpha (\epsilon_{ui} \cdot q_i - \lambda p_u)$$
$$q_i := q_i + \alpha (\epsilon_{ui} \cdot p_u - \lambda q_i)$$

where

$\epsilon_{ui}$ denotes the error between existing rating and estimated rating in the form of $r_{ui} - \hat{r}_{ui}$

$\alpha$ denotes the learning rate of gradient descent which dictates how soon the solver reaches the local optima

&nbsp;

Reference to Singular Value Decomposition

https://github.com/je-suis-tm/machine-learning/blob/master/principal%20component%20analysis.ipynb

Reference to Gradient Descent

https://github.com/je-suis-tm/machine-learning/blob/master/gradient%20descent.ipynb

Reference to Simon Funk's blog

https://sifter.org/~simon/journal/20061211.html

###### Functions

In [6]:
#use numba to dramatically boost the speed of linear algebra
#this will be a lot faster than surprise library
@numba.njit
def funk_svd_epoch(arr,miu,b_u,b_i,p_u,q_i,alpha,lambda_):
    
    #initialize
    error=0
    
    #only iterate known ratings
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            r_ui=arr[i,u]
            
            #another way to identify nan
            #r_ui!=r_ui
            if np.isnan(r_ui):
                continue

            #compute error
            epsilon_ui=r_ui-miu-b_u[u]-b_i[i]-q_i[i].T@p_u[u]
            error+=epsilon_ui**2

            #update
            b_u[u]+=alpha*(epsilon_ui-lambda_*b_u[u])
            b_i[i]+=alpha*(epsilon_ui-lambda_*b_i[i])
            p_u[u]+=alpha*(epsilon_ui*q_i[i]-lambda_*p_u[u])
            q_i[i]+=alpha*(epsilon_ui*p_u[u]-lambda_*q_i[i])
    
    return error,b_u,b_i,p_u,q_i

In [7]:
#svd inspired latent factor model by simon funk
def funk_svd(arr,miu_init=None,b_u_init=[],b_i_init=[],
             p_u_init=[],q_i_init=[],num_of_rank=40,
             alpha=0.005,lambda_=0.02,tau=0.0001,
             max_iter=20,diagnosis=True
             ):

    #initialize
    stop=False
    counter=0
    sse=None
    
    #global mean
    if not miu_init:       
        miu=arr[~np.isnan(arr)].mean()
    else:
        miu=miu_init
        
    #user baseline
    if len(b_u_init)==0:
        b_u=np.zeros(arr.shape[1])
    else:
        b_u=b_u_init
    
    #item baseline
    if len(b_i_init)==0:
        b_i=np.zeros(arr.shape[0])
    else:
        b_i=b_i_init
        
    #user latent factors
    if len(p_u_init)==0:
        p_u=np.zeros((arr.shape[1],num_of_rank))
        p_u.fill(0.1)
    else:
        p_u=p_u_init
    
    #item latent factors
    if len(q_i_init)==0:
        q_i=np.zeros((arr.shape[0],num_of_rank))
        q_i.fill(0.1)
    else:
        q_i=q_i_init
    
    #gradient descent
    while not stop:
        
        error,b_u,b_i,p_u,q_i=funk_svd_epoch(arr,miu,
                                             b_u,b_i,
                                             p_u,q_i,
                                             alpha,lambda_)

        counter+=1

        #maximum number of epoch
        if counter>=max_iter:
            stop=True
            if diagnosis:
                print('Not converged.',
                      'Consider increase number of iterations or tolerance')
                
        #use sum of squared error to determine if converged
        sse_prev=sse
        sse=error
        if sse_prev and abs(sse/sse_prev-1)<=tau:
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return b_u,b_i,p_u,q_i

###### Run

In [8]:
#initialize
num_of_latent_factors=40
max_num_of_epoch=500
learning_rate=0.01
lagrange_multiplier=0.02
tolerance=0.005

In [9]:
#funk svd
b_u,b_i,p_u,q_i=funk_svd(arr_train,num_of_rank=num_of_latent_factors,
         alpha=learning_rate,
         lambda_=lagrange_multiplier,tau=tolerance,
         max_iter=max_num_of_epoch)

30 iterations to reach convergence



In [10]:
#compute global mean
miu=arr_train[~np.isnan(arr_train)].mean()

#matrix completion
output=miu+np.repeat(
            b_u.reshape(1,-1),
            arr_train.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),arr_train.shape[1],axis=1)+q_i@p_u.T

In [11]:
#use mse as benchmark for comparison
mse_funk_svd=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('Funk SVD Mean Squared Error:',round(mse_funk_svd,3))

Funk SVD Mean Squared Error: 0.927


#### SVD++

&nbsp;

Inspired by Funk SVD, more and more variations have been invented to enhance the prediction accuracy and the convergence speed. One of the most famous variation is SVD++ (Koren, 2008). It can be seen as an augmented Funk SVD with user implicit feedback. The accuracy is further improved at the cost of implicit feedback integration. The official optimization problem is formulated as below. The notations are almost the same with only a few extra from implicit feedback.

$$ \min_{p_*,q_*,b_*,y_*}\,\sum_{r_{ui}\,\in\,\mathcal{K}} \left(r_{ui} - \mu - b_u - b_i - q_i^T\left(p_u+|N(u)|^{-\frac{1}{2}} \sum_{j \in N(u)}y_j\right) \right)^2 + \lambda\left( ||p_u||^2 + ||q_i||^2 + \sum_{j \in N(u)}||y_j||^2 + b_u^2 + b_i^2 \right)$$

where 

$N(u)$ denotes the implicit preference list of user $u$

$y_j$ denotes the item $j$ that user $u$ implicitly prefers

&nbsp;

To explain a bit, SVD++ incorporates user implicit feedback into the model. Users may have watched a lot of movies on Netflix yet they choose to rate items in $N(u)$ for a particular reason. For instance, user $u$ may have watched six seasons of House of Cards but he strongly resents the last season. Hence, he gives one star rating to the last season. This kind of implicit feedback is not captured in any latent factors. Nonetheless, the implicit feedback $y_j$ is not independent of any latent factors. Since certain combination $q_i^Tp_u$ triggers the user response and amplifies the extreme rating, $y_j$ is included in both the objective function and the constraint to avoid overfit problem. FYI, $y_j$ can be seen as another latent factor vector with the exact dimension as $q_i$.

The actual optimization solver is similar to Funk SVD with one more parameter $y_j$. 

$$b_u := b_u + \alpha (\epsilon_{ui} - \lambda b_u)$$
$$b_i := b_i + \alpha (\epsilon_{ui} - \lambda b_i)$$
$$p_u := p_u + \alpha (\epsilon_{ui} \cdot q_i - \lambda p_u)$$
$$q_i := q_i + \alpha (\epsilon_{ui} \cdot (p_u+|N(u)|^{-\frac{1}{2}} \sum_{j \in N(u)}y_j) - \lambda q_i)$$
$$y_j := y_j + \alpha (|N(u)|^{-\frac{1}{2}} \cdot \epsilon_{ui} \cdot q_i - \lambda y_j)$$

&nbsp;

Reference to the original paper of SVD++ (equation 15)

https://people.engr.tamu.edu/huangrh/Spring16/papers_course/matrix_factorization.pdf

###### Functions

In [12]:
#use numba to dramatically boost the speed of linear algebra
#this will be a lot faster than surprise library
@numba.njit
def svd_plus_plus_epoch(arr,miu,b_u,b_i,p_u,q_i,y_j,alpha,lambda_):
    
    #initialize
    error=0
    
    #only iterate known ratings
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            r_ui=arr[i,u]
            
            #another way to identify nan
            #r_ui!=r_ui
            if np.isnan(r_ui):
                continue
                
            #compute implicit feedback
            N_u=np.where(~np.isnan(arr[:,u]))[0]
            N_u_norm_sqrt=arr[:,u][~np.isnan(arr[:,u])].shape[0]**0.5
            feedback=(y_j[N_u,:]/N_u_norm_sqrt).sum(axis=0)

            #compute error
            epsilon_ui=(r_ui-miu-b_u[u]-b_i[i]-q_i[i].T@(
                feedback+p_u[u]).reshape(-1,1)).item()
            error+=epsilon_ui**2

            #update
            b_u[u]+=alpha*(epsilon_ui-lambda_*b_u[u])
            b_i[i]+=alpha*(epsilon_ui-lambda_*b_i[i])
            p_u[u]+=alpha*(epsilon_ui*q_i[i]-lambda_*p_u[u])
            q_i[i]+=alpha*(epsilon_ui*(p_u[u]+feedback)-lambda_*q_i[i])
            y_j[N_u]+=alpha*(epsilon_ui*q_i[N_u]/N_u_norm_sqrt-lambda_*y_j[N_u])
               
    return error,b_u,b_i,p_u,q_i,y_j

In [13]:
#svdpp latent factor model
def svd_plus_plus(arr,miu_init=None,b_u_init=[],b_i_init=[],
             p_u_init=[],q_i_init=[],y_j_init=[],num_of_rank=40,
             alpha=0.005,lambda_=0.02,tau=0.0001,
             max_iter=20,diagnosis=True
             ):

    #initialize
    stop=False
    counter=0
    sse=None
    
    #global mean
    if not miu_init:       
        miu=arr[~np.isnan(arr)].mean()
    else:
        miu=miu_init
        
    #user baseline
    if len(b_u_init)==0:
        b_u=np.zeros(arr.shape[1])
    else:
        b_u=b_u_init
    
    #item baseline
    if len(b_i_init)==0:
        b_i=np.zeros(arr.shape[0])
    else:
        b_i=b_i_init
        
    #user latent factors
    if len(p_u_init)==0:
        p_u=np.zeros((arr.shape[1],num_of_rank))
        p_u.fill(0.1)
    else:
        p_u=p_u_init
    
    #item latent factors
    if len(q_i_init)==0:
        q_i=np.zeros((arr.shape[0],num_of_rank))
        q_i.fill(0.1)
    else:
        q_i=q_i_init
        
    #user implicit feedback
    if len(y_j_init)==0:
        y_j=np.zeros((arr.shape[0],num_of_rank))
        y_j.fill(0.1)
    else:
        y_j=y_j_init
    
    #gradient descent
    while not stop:
        
        error,b_u,b_i,p_u,q_i,y_j=svd_plus_plus_epoch(
            arr,miu,b_u,b_i,p_u,q_i,y_j,alpha,lambda_)

        counter+=1
        
        #maximum number of epoch
        if counter>=max_iter:
            stop=True
            if diagnosis:
                print('Not converged.',
                      'Consider increase number of iterations or tolerance')
                
        #use sum of squared error to determine if converged
        sse_prev=sse
        sse=error
        if sse_prev and abs(sse/sse_prev-1)<=tau:
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return b_u,b_i,p_u,q_i,y_j

###### Run

In [14]:
#svdpp
b_u,b_i,p_u,q_i,y_j=svd_plus_plus(arr_train,
         num_of_rank=num_of_latent_factors,
         alpha=learning_rate,
         lambda_=lagrange_multiplier,tau=tolerance,
         max_iter=max_num_of_epoch)

17 iterations to reach convergence



In [15]:
#compute implicit feedback
feedback=[]
for u in range(arr_train.shape[1]):
    N_u=np.where(~np.isnan(arr_train[:,u]))[0]
    N_u_norm_sqrt=arr_train[:,u][~np.isnan(arr_train[:,u])].shape[0]**0.5
    feedback.append((y_j[N_u,:]/N_u_norm_sqrt).sum(axis=0))

In [16]:
#matrix completion
output=miu+np.repeat(
            b_u.reshape(1,-1),
            arr_train.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),
    arr_train.shape[1],axis=1)+q_i@(p_u+np.mat(feedback)).T

In [17]:
#use mse as benchmark for comparison
mse_svdpp=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('SVD++ Mean Squared Error:',round(mse_svdpp,3))

SVD++ Mean Squared Error: 0.921


#### Non-negative Matrix Factorization

&nbsp;

Funk SVD is cool but not without caveats. Sometimes nonnegativity is required to better interpret the causality of the latent variables. This is where Non-negative Matrix Factorization comes to the theatre. For simplicity, the official optimization problem is formulated with two latent variables only, $p_u$ and $q_i$.

$$ \min_{p_*,q_*}\,\sum_{r_{ui}\,\in\,\mathcal{K}} \left(r_{ui} - q_i^Tp_u \right)^2 + \lambda( ||p_u||^2 + ||q_i||^2 )$$

&nbsp;

Intuitively, we can derive the gradient descent update rule via $\theta:=\theta-\alpha\frac{\partial J(\theta)}{\partial \theta}$.

$$p_u := p_u + \alpha ( \left( r_{ui} - \hat{r}_{ui} \right) \cdot q_i - \lambda p_u)$$

$$q_i := q_i + \alpha ( \left( r_{ui} - \hat{r}_{ui} \right) \cdot p_u - \lambda q_i)$$

&nbsp;

The issue with a conventional gradient descent is that it cannot guarantee the non-negativity. Hence, we introduce a modified version called Multiplicative Update. MU can be derived from the learning rate $\alpha$. Take user preference $p_u$ for example, if we set learning rate $\alpha = \frac {p_u}{\hat{r}_{ui} \cdot q_i + \lambda p_u}$. We get rid of the subtraction and obtain an update rule based upon multiplication.

$$p_u := p_u \frac {r_{ui} \cdot q_i}{\hat{r}_{ui} \cdot q_i + \lambda p_u}$$

Similarly, if we set learning rate $\alpha = \frac {q_i}{\hat{r}_{ui} \cdot p_u + \lambda q_i}$ for item attribute $q_i$, we can obtain another set of MU.

$$q_i := q_i \frac {r_{ui} \cdot p_u}{\hat{r}_{ui} \cdot p_u + \lambda q_i}$$

&nbsp;

Quite a few existing packages provide NMF. Apart from `surprise.prediction_algorithms.matrix_factorization.NMF`, you can actually find NMF in `sklearn.decomposition.NMF`.

A good tutorial on NMF

https://perso.telecom-paristech.fr/essid/teach/NMF_tutorial_ICME-2014.pdf

Reference to the original paper of MU on NMF

https://proceedings.neurips.cc/paper/2000/file/f9d1152547c0bde01830b7e8bd60024c-Paper.pdf

###### Functions

In [18]:
#use numba to dramatically boost the speed of linear algebra
#this will be a lot faster than surprise library
@numba.njit
def nmf_epoch(arr,p_u,q_i,lambda_):
    
    #initialize
    error=0
    
    #only iterate known ratings
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            r_ui=arr[i,u]
            num_of_users=len(np.where(~np.isnan(arr[i]))[0])
            num_of_items=len(np.where(~np.isnan(arr[:,u]))[0])
            
            #another way to identify nan
            #r_ui!=r_ui
            if np.isnan(r_ui):
                continue

            #compute error
            estimated=q_i[i].T@p_u[u]
            epsilon_ui=r_ui-estimated
            error+=epsilon_ui**2

            #update
            p_u[u]=np.multiply(p_u[u],np.divide(q_i[i]*r_ui,
                             q_i[i]*estimated+lambda_*p_u[u]*num_of_items))
            q_i[i]=np.multiply(q_i[i],np.divide(p_u[u]*r_ui,
                             p_u[u]*estimated+lambda_*q_i[i]*num_of_users))
    
    return error,p_u,q_i

In [19]:
#nmf latent factor model
def nmf(arr,p_u_init=[],q_i_init=[],num_of_rank=40,
             lambda_=0.02,tau=0.0001,
             max_iter=20,diagnosis=True
             ):

    #initialize
    stop=False
    counter=0
    sse=None
    
    #user latent factors
    if len(p_u_init)==0:
        p_u=np.zeros((arr.shape[1],num_of_rank))
        p_u.fill(0.1)
    else:
        p_u=p_u_init
    
    #item latent factors
    if len(q_i_init)==0:
        q_i=np.zeros((arr.shape[0],num_of_rank))
        q_i.fill(0.1)
    else:
        q_i=q_i_init
        
    #multiplicative update
    while not stop:
        
        error,p_u,q_i=nmf_epoch(arr,p_u,q_i,lambda_)

        counter+=1
        
        #maximum number of epoch
        if counter>=max_iter:
            stop=True
            if diagnosis:
                print('Not converged.',
                      'Consider increase number of iterations or tolerance')
                
        #use sum of squared error to determine if converged
        sse_prev=sse
        sse=error
        if sse_prev and abs(sse/sse_prev-1)<=tau:
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return p_u,q_i

###### Run

In [20]:
#nmf is highly sensitive to initial values
p_u_nmf_init=np.random.uniform(size=(arr.shape[1],
                            num_of_latent_factors))
q_i_nmf_init=np.random.uniform(size=(arr.shape[0],
                            num_of_latent_factors))

In [21]:
#nmf
p_u,q_i=nmf(arr_train,p_u_init=p_u_nmf_init,q_i_init=q_i_nmf_init,
            num_of_rank=num_of_latent_factors,
            lambda_=lagrange_multiplier,
            tau=tolerance,
            max_iter=max_num_of_epoch)

6 iterations to reach convergence



In [22]:
#matrix completion
output=q_i@p_u.T

In [23]:
#use mse as benchmark for comparison
mse_nmf=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('NMF Mean Squared Error:',round(mse_nmf,3))

NMF Mean Squared Error: 6.987


#### Co-clustering

&nbsp;

Co-clustering is an unsupervised multi-label classification in a 2D matrix. Each element in the matrix depends on two labels – row cluster and column cluster. The training strategy is simply one-versus-rest, meaning the user cluster label and item cluster label of each element are trained separately by K Means. Initially 
$K_u$ user centroids and $K_i$ item centroids are randomly assigned to each rating $r_{ui}$. Repeat the following steps until convergence. The convergence can be defined as maximum number of iterations, the fixed position of each cluster or the relative error change.

* E-step: compute the distance from each data point $r_{ui}$ to $K_u$ and $K_i$ centroids. Each label of the data point is determined by the centroid with the shortest distance. 

* M-step: Each centroid $C_*$ is recalibrated to the mean of all the data points $r_{ui}$ which fall under its jurisdiction.

&nbsp;

In the context of recommender system, the distance metric is defined as the squared error between estimated rating and actual rating. The estimated rating is influenced by the user mean $\mu_u$ (think of it as user baseline), the item mean $\mu_i$ (think of it as item baseline) and the co-clustering effect $\overline{C_{ui}}$. One thing to bear in mind is that the user clustering effect $\overline{C_{u}}$ contributes to both user mean and co-clustering effect. It has to be deducted from the estimated rating to avoid double counting. The same applies to the item clustering effect $\overline{C_{i}}$.

$$distance(r_{ui},C_*) = \left( r_{ui}- \hat{r}_{ui} \right)^2 = \left( r_{ui} - \overline{C_{ui}} - (\mu_u - \overline{C_u}) - (\mu_i - \overline{C_i}) \right)^2$$

where 

$\overline{C_{ui}}$ denotes the average rating of co-cluster $C_{ui}$

$\overline{C_u}$ denotes the average rating of user cluster $C_u$

$\overline{C_i}$ denotes the average rating of item  cluster $C_i$

&nbsp;

Reference to K Means

https://github.com/je-suis-tm/machine-learning/blob/master/k%20means.ipynb

Reference to the original paper by Thomas George and Srujana Merugu

https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.113.6458&rep=rep1&type=pdf

###### Functions

In [24]:
#compute mean of user,item and co-cluster
@numba.njit
def compute_avg(arr,num_of_cluster_item,
           num_of_cluster_user,cluster_item,cluster_user):
    
    #compute co-cluster mean
    bar_c_ui=np.zeros((num_of_cluster_item,
                       num_of_cluster_user))
    for c_i in range(num_of_cluster_item):
        for c_u in range(num_of_cluster_user):
            row_id=np.where(cluster_item==c_i)[0]
            col_id=np.where(cluster_user==c_u)[0]
            cocluster_sub=arr[row_id][:,col_id].copy().flatten()
            bar_c_ui[c_i][c_u]=(cocluster_sub[~np.isnan(cocluster_sub)]).mean()
            
    #compute user and item mean
    miu_i=[]
    for i in range(arr.shape[0]):
        subset=arr[i][~np.isnan(arr[i])]
        if len(subset)==0:
            miu_i.append(np.nan)
        else:
            miu_i.append(subset.mean())
    miu_u=[]
    for u in range(arr.shape[1]):
        subset=arr[:,u][~np.isnan(arr[:,u])]
        if len(subset)==0:
            miu_u.append(np.nan)
        else:
            miu_u.append(subset.mean())
    
    return bar_c_ui,miu_i,miu_u

In [25]:
#compute mean of user and item cluster
@numba.njit
def m_step(arr,num_of_cluster_item,
           num_of_cluster_user,cluster_item,
           cluster_user):

    #compute mean of item cluster
    bar_cluster_item=[]
    for c_i in range(num_of_cluster_item):
        cluster_sub=arr[np.where(cluster_item==c_i)[0]].copy().flatten()
        bar_cluster_item.append((cluster_sub[~np.isnan(cluster_sub)]).mean())

    #compute mean of user cluster
    bar_cluster_user=[]
    for c_u in range(num_of_cluster_user):
        cluster_sub=arr[:,np.where(cluster_user==c_u)[0]].copy().flatten()
        bar_cluster_user.append((cluster_sub[~np.isnan(cluster_sub)]).mean())
    
    #compute mean of user,item and co-cluster
    bar_c_ui,miu_i,miu_u=compute_avg(arr,num_of_cluster_item,
                       num_of_cluster_user,cluster_item,cluster_user)
    
    return bar_c_ui,miu_i,miu_u,bar_cluster_item,bar_cluster_user

In [26]:
#reassign user cluster
@numba.njit
def get_cluster_user(arr,bar_c_ui,miu_i,miu_u,num_of_cluster_item,
           num_of_cluster_user,cluster_item,cluster_user,
           bar_cluster_item,bar_cluster_user):
    
    #iterate through users
    for u in range(arr.shape[1]):
          
        #get user cluster
        sorting=[]
        for c_u in range(num_of_cluster_user):
            sorting.append(0)
            for i in np.where(~np.isnan(arr[:,u]))[0]:            
                c_i=cluster_item[i]
                r_ui=arr[i,u]
    
                #compute estimated
                if np.isnan(miu_u[u]):
                    user_part=0
                else:
                    user_part=miu_u[u]-bar_cluster_user[c_u]
                if np.isnan(miu_i[i]):
                    item_part=0
                else:
                    item_part=miu_i[i]-bar_cluster_item[c_i]
                if np.isnan(bar_c_ui[c_i,c_u]):
                    co_part=arr.flatten()[~np.isnan(arr.flatten())].mean()
                else:
                    co_part=bar_c_ui[c_i,c_u]
                est=co_part+user_part+item_part
    
                #compute sum of squared error
                sorting[-1]+=(r_ui-est)**2
        
        #assign to the user cluster that minimizes sse
        cluster_user[u]=sorting.index(list(set(sorting))[0])
    
    return cluster_user

In [27]:
#reassign item cluster
@numba.njit
def get_cluster_item(arr,bar_c_ui,miu_i,miu_u,num_of_cluster_item,
           num_of_cluster_user,cluster_item,cluster_user,
           bar_cluster_item,bar_cluster_user):
    
    #iterate through items
    for i in range(arr.shape[0]):
            
        #get item cluster
        sorting=[]
        for c_i in range(num_of_cluster_item):
            sorting.append(0)
            for u in np.where(~np.isnan(arr[i]))[0]: 
                c_u=cluster_user[u]
                r_ui=arr[i,u]
    
                #compute estimated
                if np.isnan(miu_u[u]):
                    user_part=0
                else:
                    user_part=miu_u[u]-bar_cluster_user[c_u]
                if np.isnan(miu_i[i]):
                    item_part=0
                else:
                    item_part=miu_i[i]-bar_cluster_item[c_i]
                if np.isnan(bar_c_ui[c_i,c_u]):
                    co_part=arr.flatten()[~np.isnan(arr.flatten())].mean()
                else:
                    co_part=bar_c_ui[c_i,c_u]
                est=co_part+user_part+item_part
                
                #compute sum of squared error
                sorting[-1]+=(r_ui-est)**2
        
        #assign to the item cluster that minimizes sse
        cluster_item[i]=sorting.index(list(set(sorting))[0])
    
    return cluster_item

In [28]:
#reassign user and item clusters
@numba.njit
def e_step(arr,bar_c_ui,miu_i,miu_u,
           num_of_cluster_item,num_of_cluster_user,
           cluster_item,cluster_user,
           bar_cluster_item,bar_cluster_user):

    cluster_user=get_cluster_user(arr,bar_c_ui,miu_i,miu_u,
                                  num_of_cluster_item,num_of_cluster_user,
                                  cluster_item,cluster_user,
                                   bar_cluster_item,bar_cluster_user)

    cluster_item=get_cluster_item(arr,bar_c_ui,miu_i,miu_u,
                                  num_of_cluster_item,num_of_cluster_user,
                                  cluster_item,cluster_user,
                                   bar_cluster_item,bar_cluster_user)
    
    return cluster_item,cluster_user

In [29]:
#matrix completion
@numba.njit
def cocluster_predict(arr,bar_c_ui,miu_i,miu_u,bar_cluster_item,
                      bar_cluster_user,cluster_item,cluster_user):

    #map cocluster mean into array shape
    cluster_mapping=[bar_c_ui[
        cluster_item[i],cluster_user[u]] for i in range(
        arr.shape[0]) for u in range(arr.shape[1])]
    cocluster_mean=np.array(cluster_mapping)

    #fill na with global mean
    cocluster_mean[np.isnan(cocluster_mean)]=arr.flatten()[
        ~np.isnan(arr.flatten())].mean()
    copart=cocluster_mean.reshape(arr.shape)
    
    #map item mean into array shape
    item_cluster_mean=np.array(
        [bar_cluster_item[c_i] for c_i in cluster_item for _ in range(
            arr.shape[1])])    
    item_mean=np.array(
        [bar_i for bar_i in miu_i for _ in range(
            arr.shape[1])])

    #compute
    item_part=item_mean-item_cluster_mean

    #fill na with zero
    item_part[np.isnan(item_part)]=0
    item_part=item_part.reshape(arr.shape)

    #map user mean into array shape
    user_cluster_mean=np.array([bar_cluster_user[
                c_u] for c_u in cluster_user]*arr.shape[0])
    user_mean=np.array(miu_u*arr.shape[0])           

    #compute
    user_part=user_mean-user_cluster_mean

    #fill na with zero
    user_part[np.isnan(user_part)]=0
    user_part=user_part.reshape(arr.shape)
    
    #forecast
    est=copart+user_part+item_part
    
    return est

In [30]:
#co-clustering model
def coclustering(arr,cluster_user_init=[],cluster_item_init=[],
                 num_of_cluster_user=3,
                 num_of_cluster_item=3,tau=0.0001,
                 max_iter=200,diagnosis=True
                 ):

    #initialize
    stop=False
    counter=0
    sse=None
    
    #initialize item cluster
    if len(cluster_item_init)==0:
        cluster_item=np.random.choice(
            range(num_of_cluster_item),arr.shape[0])
    else:
        cluster_item=cluster_item_init
    
    #initialize user cluster
    if len(cluster_user_init)==0:
        cluster_user=np.random.choice(
            range(num_of_cluster_user),arr.shape[1])
    else:
        cluster_user=cluster_user_init
            
    #compute mean of user and item cluster
    bar_c_ui,miu_i,miu_u,bar_cluster_item,bar_cluster_user=m_step(
            arr,num_of_cluster_item,num_of_cluster_user,cluster_item,
                   cluster_user)
    
    #keep track of cluster assignment
    cluster_item_prev=cluster_item.copy()
    cluster_user_prev=cluster_user.copy()
    
    #k means
    while not stop:
        
        #e step
        cluster_item,cluster_user=e_step(arr,bar_c_ui,miu_i,miu_u,
                           num_of_cluster_item,num_of_cluster_user,
                           cluster_item,cluster_user,
                           bar_cluster_item,bar_cluster_user)

        #m step
        bar_c_ui,miu_i,miu_u,bar_cluster_item,bar_cluster_user=m_step(
            arr,num_of_cluster_item,num_of_cluster_user,cluster_item,
                   cluster_user)

        counter+=1
        
        #maximum number of epoch
        if counter>=max_iter:
            stop=True
            if diagnosis:
                print('Not converged.',
                      'Consider increase number of iterations',
                      'or change number of clusters')
                
        #check cluster assignment to determine if converged
        if (cluster_item_prev==cluster_item).all() and \
        (cluster_user_prev==cluster_user).all():
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')
        
        #use sum of squared error to determine if converged
        estimated=cocluster_predict(arr,bar_c_ui,
                                 miu_i,miu_u,bar_cluster_item,
                                  bar_cluster_user,
                                 cluster_item,cluster_user)
        sse_prev=sse
        sse=np.square(arr-estimated).sum()
        if sse_prev and abs(sse/sse_prev-1)<=tau:
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return bar_c_ui,miu_i,miu_u,bar_cluster_item, \
bar_cluster_user,cluster_item,cluster_user

###### Run

In [31]:
#initialize the number of clusters
num_c_u=3
num_c_i=3

In [32]:
#co-clustering
bar_c_ui,miu_i,miu_u,bar_cluster_item,bar_cluster_user, \
cluster_item,cluster_user=coclustering(
             arr_train,num_of_cluster_user=num_c_u,
             num_of_cluster_item=num_c_i,
             tau=tolerance,
             max_iter=max_num_of_epoch
                 )

Not converged. Consider increase number of iterations or change number of clusters


In [33]:
#matrix completion
output=cocluster_predict(arr_train,bar_c_ui,miu_i,miu_u,bar_cluster_item,
                      bar_cluster_user,cluster_item,cluster_user)

In [34]:
#use mse as benchmark for comparison
mse_cocluster=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('Co-clustering Mean Squared Error:',round(mse_cocluster,3))

Co-clustering Mean Squared Error: 0.983


### Memory-based

#### Slope One

&nbsp;

Slope one is probably the easiest memory-based method. The name comes from the naïve form of $f(x) = kx + b$. In this particular version, slope $k$ is set at $1$, hence the name `Slope One`. The intercept $b$ is defined as the user mean $\mu_u$ and the variable $x$ is defined as the average deviation of relevant items. By using the previous notations, we are able to obtain the following equation of slope one.

$$\hat{r}_{ui} = \frac{1}{|R_i(u)|}
        \sum\limits_{j \in R_i(u)} \text{dev}(i, j) + \mu_u$$

where

$\mu_u$ denotes the average rating by user $u$

$R_i(u)$ denotes the set of relevant items which both user $u$ and user $v$, who has rated item $i$, have rated

$\text{dev}_(i, j)$ denotes the average rating deviation between item $i$ and relevant item $j$ rated by users who have rated item $i$

&nbsp;

By the definition of $\text{dev}_(i, j)$, the average deviation of relevant items can be thought as the relative item baseline $b_i$ which explains why item $i$ attracts higher/lower score than item $j$ across all users. The equation is formulated below.

$$\text{dev}(i, j) = \frac{1}{        |U_{ij}|}\sum\limits_{u \in U_{ij}} r_{ui} - r_{uj}$$

where

$U_{ij}$ denotes the set of users who have rated both item $i$ and relevant item $j$

&nbsp;

Reference to the original paper by Daniel Lemire and Anna Maclachlan

https://www.researchgate.net/publication/1960789_Slope_One_Predictors_for_Online_Rating-Based_Collaborative_Filtering

###### Functions

In [35]:
#basic slope one algorithm
@numba.njit
def slope_one(arr,miu):
                
    #create a copy
    estimated=arr.copy()
        
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            if np.isnan(arr[i,u]):
                
                #get user mean
                miu_u=(arr[:,u][~np.isnan(arr[:,u])]).mean()
                
                #take global mean for cold start
                if np.isnan(miu_u):
                    miu_u=miu
                
                #items rated by user u
                items_rated_u=set(np.where(~np.isnan(arr[:,u]))[0])

                #users who have rated item i
                users_rated_i=np.where(~np.isnan(arr[i]))[0]
                
                #items rated by users who have rated item i
                items_rated_others=set(np.where(~np.isnan(
                    arr_train[:,users_rated_i]))[0])
                    
                #find relevant items
                R_i_u=items_rated_u.intersection(items_rated_others)

                #take user mean if user u share nothing in common with others
                if len(R_i_u)==0:                    
                    estimated[i,u]=miu_u
                else:

                    #compute average deviation of relevant items
                    relevant_items=[]
                    for j in R_i_u:
                                                   
                        #find users who have rated both i and j
                        U_i_j=set(users_rated_i).intersection(
                            set(np.where(~np.isnan(arr[j]))[0]))

                        #compute average deviation 
                        #for the users who have rated both i and j
                        dev_i_j=np.mean(arr[i][
                            np.array(list(U_i_j))]-arr[j][
                            np.array(list(U_i_j))])
                        relevant_items.append(dev_i_j)
                    
                    #predict
                    estimated[i,u]=miu_u+np.mean(np.array(relevant_items))
                    
    return estimated

###### Run

In [36]:
#matrix completion
output=slope_one(arr_train,miu)

In [37]:
#use mse as benchmark for comparison
mse_slop_one=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('Slope One Mean Squared Error:',round(mse_slop_one,3))

Slope One Mean Squared Error: 0.985


#### K Nearest Neighbors

&nbsp;

KNN is undoubtedly an easy supervised learning model. In the iris dataset, we have shown how KNN works in classification problem. In the case of recommender system, it is effectively a regression problem. We have to slightly change the methodology. 

* KNN in recommender system takes the weighted average of neighbors to predict the rating, albeit KNN in the iris dataset takes the majority label of neighbors to predict the classification.

* KNN in recommender system uses cosine similarity, mean squared distance similarity or Pearson correlation similarity to define neighbors, whereas KNN in the iris dataset uses Euclidean distance, Chebyshev distance or Manhattan distance to define neighbors.

In KNN, you also have the choice of picking neighbors based upon the users (how similar users rate the same item) or the items (how similar items get rated by the same user). In this script, every neighborhood model will be based upon the users. For item approach, just switch user and item in the equation below.

$$\hat{r}_{ui} = \frac{  \sum_{v \in N^k_i(u)} \mathcal{similarity}(u, v) \cdot r_{vi}}  {\sum_{v \in N^k_i(u)} \mathcal{similarity}(u, v)}$$

where 

$v$ denotes the neighbors of user $u$

$N^k_i(u)$ denotes the set of top $k$ neighbors of user $u$ that have rated item $i$

$\mathcal{similarity}(u, v)$ denotes the similarity metric between user $u$ and its neighbor $v$

&nbsp;

Each similarity metric has its own pros and cons. In this case, we will pick the MSD similarity which is formulated below. The biggest selling point of MSD is its resemblance to Euclidean distance. One of its malaise is it doesn't penalize a small number of $|I_{uv}|$. A sum of squared error of 0 with $|I_{uv}|$ at 1 is inevitably inferior to a sum of squared error of 1 with $|I_{uv}|$ at 10 but the latter yields a smaller MSD even the latter pair has rated more items in common.

$$\mathcal{similarity}(u, v) = \frac{1}{\frac{\sum_{i \in I_{uv}} (r_{ui} - r_{vi})^2}{|I_{uv}|}
         + 1}$$

where 

$I_{uv}$ denotes the set of items where both user $u$ and its neighbor $v$ have rated

&nbsp;

Reference to KNN in iris dataset

https://github.com/je-suis-tm/machine-learning/blob/master/k%20nearest%20neighbors.ipynb

Reference to cosine similarity

https://github.com/je-suis-tm/machine-learning/blob/master/latent%20semantic%20indexing.ipynb

###### Functions

In [38]:
#obtain msd similarity matrix
@numba.njit
def get_msd_similarity_matrix(arr):

    similarity_matrix=np.zeros((arr.shape[1],arr.shape[1]))
    for u in range(arr.shape[1]):
        for v in range(u+1,arr.shape[1]):
            
            #self correlation distorts knn selection
            if u==v:
                continue

            #compute msd first then eliminate nan
            I_uv=np.square(arr[:,u]-arr[:,v])
            valid_I_uv=I_uv[~np.isnan(I_uv)]

            #avoid the case where two users havent rated any items in common
            if len(valid_I_uv)>0:
                msd=1/(valid_I_uv.sum()/len(valid_I_uv)+1)
            else:
                msd=0

            #symmetric matrix
            similarity_matrix[u,v]=msd
            similarity_matrix[v,u]=msd
            
    return similarity_matrix

In [39]:
#knn for matrix completion
@numba.njit
def knn(arr,similarity_matrix,top_k=10):

    estimated=arr.copy()
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            if np.isnan(arr[i,u]):

                #find top k neighbor based upon similarity matrix
                rated_users=np.where(~np.isnan(arr[i]))[0]
                similarities=similarity_matrix[u][rated_users]
                top_k_neighbors=np.argsort(similarities)[-top_k:]
                N_k_i_u=np.array([
                    rated_users[neighbor] for neighbor in top_k_neighbors])

                #compute weighted average                
                if len(N_k_i_u)!=0:
                    numerator=arr[i][N_k_i_u].T@similarity_matrix[u][N_k_i_u]
                    denominator=similarity_matrix[u][N_k_i_u].sum()
                    if denominator!=0:
                        estimated[i,u]=numerator/denominator
                        continue
                        
                #when the users who rated item i
                #have nothing in common with user u
                #take the average of user mean and item mean
                #if both user mean and item mean are empty
                #take global mean
                miu_i=arr[i][~np.isnan(arr[i])]
                miu_u=arr[:,u][~np.isnan(arr[:,u])]                
                if len(miu_i)==0 and len(miu_u)==0:
                    estimated[i,u]=arr[np.where(~np.isnan(arr))[0]][
                        np.where(~np.isnan(arr))[1]].mean()
                else:
                    estimated[i,u]=np.array(list(miu_i)+list(miu_u)).mean()
                    
    return estimated

###### Run

In [40]:
top_k=10

In [41]:
#compute similarity matrix
similarity_matrix=get_msd_similarity_matrix(arr_train)

#matrix completion
output=knn(arr_train,similarity_matrix,top_k=top_k)

In [42]:
#use mse as benchmark for comparison
mse_knn=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('KNN Mean Squared Error:',round(mse_knn,3))

KNN Mean Squared Error: 0.986


#### K Nearest Neighbors with mean

&nbsp;

Now that we are familiar with KNN, it's about time to introduce many of its variations. The first one is KNN with mean. In the user approach, it takes consideration of user mean and neighbor mean into the equation.

$$\hat{r}_{ui} = \mu_u + \frac{  \sum_{v \in N^k_i(u)} \mathcal{similarity}(u, v) \cdot (r_{vi}-\mu_v)}  {\sum_{v \in N^k_i(u)} \mathcal{similarity}(u, v)}$$

where 

$\mu_u$ denotes the average rating of user $u$

$\mu_v$ denotes the average rating of neighbor $v$

###### Functions

In [43]:
#knn with mean for matrix completion
@numba.njit
def knn_with_mean(arr,similarity_matrix,top_k=10):
    
    #compute user mean
    user_mean=np.array([arr[:,u][
        ~np.isnan(arr[:,u])].mean() for u in range(arr.shape[1])])

    estimated=arr.copy()
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            if np.isnan(arr[i,u]):

                #find top k neighbor based upon similarity matrix
                rated_users=np.where(~np.isnan(arr[i]))[0]
                similarities=similarity_matrix[u][rated_users]
                top_k_neighbors=np.argsort(similarities)[-top_k:]
                N_k_i_u=np.array([
                    rated_users[neighbor] for neighbor in top_k_neighbors])

                #compute weighted average                
                if len(N_k_i_u)!=0:
                    numerator=(arr[i][
                        N_k_i_u]-user_mean[
                        N_k_i_u]).T@similarity_matrix[u][N_k_i_u]
                    denominator=similarity_matrix[u][N_k_i_u].sum()
                    if denominator!=0:
                        estimated[i,u]=user_mean[u]+numerator/denominator
                        continue
                        
                #when the users who rated item i
                #have nothing in common with user u
                #take the average of user mean and item mean
                #if both user mean and item mean are empty
                #take global mean
                miu_i=arr[i][~np.isnan(arr[i])]
                miu_u=arr[:,u][~np.isnan(arr[:,u])]                
                if len(miu_i)==0 and len(miu_u)==0:
                    estimated[i,u]=arr[np.where(~np.isnan(arr))[0]][
                        np.where(~np.isnan(arr))[1]].mean()
                else:
                    estimated[i,u]=np.array(list(miu_i)+list(miu_u)).mean()
                    
    return estimated

###### Run

In [44]:
#matrix completion
output=knn_with_mean(arr_train,similarity_matrix,top_k=top_k)

In [45]:
#use mse as benchmark for comparison
mse_knn_with_miu=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('KNN with μ Mean Squared Error:',round(mse_knn_with_miu,3))

KNN with μ Mean Squared Error: 0.971


#### K Nearest Neighbors with z score

&nbsp;

On top of KNN with mean, we can include the standard deviation of user into the equation. After all, some users may be moody so their ratings tend to have a larger fluctuation.

$$\hat{r}_{ui} = \mu_u + \sigma_u \cdot \frac{  \sum_{v \in N^k_i(u)} \mathcal{similarity}(u, v) \cdot \frac {(r_{vi}-\mu_v)} {\sigma_v}  }  {\sum_{v \in N^k_i(u)} \mathcal{similarity}(u, v) } $$

where 

$\sigma_u$ denotes the standard deviation of the rating from user $u$

$\sigma_v$ denotes the standard deviation of the rating from neighbor $v$

###### Functions

In [46]:
#knn with z score for matrix completion
@numba.njit
def knn_with_zscore(arr,similarity_matrix,top_k=10):
    
    #compute user mean and std
    user_mean=np.array([arr[:,u][
        ~np.isnan(arr[:,u])].mean() for u in range(arr.shape[1])])
    user_std=np.array([arr[:,u][
        ~np.isnan(arr[:,u])].std() for u in range(arr.shape[1])])

    estimated=arr.copy()
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            if np.isnan(arr[i,u]):

                #find top k neighbor based upon similarity matrix
                rated_users=np.where(~np.isnan(arr[i]))[0]
                similarities=similarity_matrix[u][rated_users]
                top_k_neighbors=np.argsort(similarities)[-top_k:]
                N_k_i_u=np.array([
                    rated_users[neighbor] for neighbor in top_k_neighbors])

                #compute weighted average                
                if len(N_k_i_u)!=0:
                    z_score=np.divide((arr[i][N_k_i_u]-user_mean[N_k_i_u]),
                                      user_std[N_k_i_u])
                    numerator=z_score.T@similarity_matrix[u][N_k_i_u]
                    denominator=similarity_matrix[u][N_k_i_u].sum()
                    if denominator!=0:
                        estimated[i,u]=user_mean[u]+user_std[
                            u]*numerator/denominator
                        continue
                        
                #when the users who rated item i
                #have nothing in common with user u
                #take the average of user mean and item mean
                #if both user mean and item mean are empty
                #take global mean
                miu_i=arr[i][~np.isnan(arr[i])]
                miu_u=arr[:,u][~np.isnan(arr[:,u])]                
                if len(miu_i)==0 and len(miu_u)==0:
                    estimated[i,u]=arr[np.where(~np.isnan(arr))[0]][
                        np.where(~np.isnan(arr))[1]].mean()
                else:
                    estimated[i,u]=np.array(list(miu_i)+list(miu_u)).mean()
                    
    return estimated

###### Run

In [47]:
#matrix completion
output=knn_with_zscore(arr_train,similarity_matrix,top_k=top_k)

In [48]:
#use mse as benchmark for comparison
mse_knn_with_zscore=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('KNN with Z Score Mean Squared Error:',round(mse_knn_with_zscore,3))

KNN with Z Score Mean Squared Error: 0.997


### Hybird

#### K Nearest Neighbors with baseline

##### Baseline Model via ALS

&nbsp;

Before we talk about KNN with baseline, it is necessarily to talk about the baseline model. Even though we have given extensive coverage of baseline in Funk SVD, KNN does not really do any gradient descent. Thus, the baseline involved in KNN has to be estimated separately. The baseline model is formulated below. It is merely Funk SVD without latent factors.

$$ \min_{b_*}\,\sum_{r_{ui}\,\in\,\mathcal{K}} \left(r_{ui} - \mu - b_u - b_i \right)^2 + \lambda( b_u^2 + b_i^2 )$$

In gradient descent, the equation can be solved via

$$b_u := b_u + \alpha (\epsilon_{ui} - \lambda b_u)$$
$$b_i := b_i + \alpha (\epsilon_{ui} - \lambda b_i)$$

However, we intend to introduce another method called Alternating Least Square. The way it works is similar to coordinate descent. At each epoch, we estimate one parameter and fix the rest. The haute couture of ALS is its ability to estimate each parameter in parallel. It can execute a lot quicker than GD in high dimensional data. Another benefit is we do not need to tune the learning rate of GD. The derivation of ALS is formulated below.

Let's start to estimate $b_u$. The term $r_{ui}-\mu-b_i$ can be considered as a constant $\phi$. Yet, in a standard least square problem, the equation is in the form of $y=x \cdot \beta^T$. We have to assume $b_u$ follows the form of $b_u \cdot I^T$.

$$ J(b_u)= \left(\phi - b_u \cdot I^T \right)^{T} \left(\phi - b_u \cdot I^T \right) + \lambda( b_u^2 + b_i^2 )$$

$$\frac{\partial J(b_u)}{\partial\,b_u}=2 I^T \left(\phi - b_u \cdot I^T \right)-2\lambda b_u=0$$

By using $I \cdot b_u^T=b_u \cdot I^T$, we obtain

$$\frac{\partial J(b_u)}{\partial\,b_u}=I^T \phi -I^T I \cdot b_u^T - \lambda b_u=0$$

$$ b_u := \left( I^T I + \lambda \right) ^ {-1} I^T \phi $$

Now $b_u$ can be estimated from a Ridge Regression estimator. In our case, $I$ is a vector of ones. $b_u \cdot I^T$ is an operation which repeats $b_u$ into the dimension of a full matrix $r$ (or $\phi$). $I^T I$ is normally the length of vector $b_u$ (a scalar) but $r$ is an incomplete matrix so $I^T I$ denotes a vector that contains the number of users which rated item $i$. $I^T \phi$ is merely the summation of $r_{ui}-\mu-b_i$. Similarly, we define $\psi=r_{ui}-\mu-b_u$ and $H$ is a vector of ones. We can easily derive the update rule for $b_i$ as well.

$$ b_i := \left( H^T H + \lambda \right) ^ {-1} H^T \psi $$

&nbsp;

Reference to ALS

https://solomonik.cs.illinois.edu/teaching/cs554/slides/slides_15.pdf

Reference to Coordinate Descent

https://github.com/je-suis-tm/machine-learning/blob/master/coordinate%20descent%20for%20elastic%20net.ipynb

###### Functions

In [49]:
#use numba to dramatically boost the speed of linear algebra
#this will be a lot faster than surprise library
@numba.njit
def als_epoch(arr,miu,b_u,b_i,lambda_):
    
    #initialize
    error=0
    
    #avoid np repeat to use numba
    #equivalent to np.repeat(b_u.reshape(1,-1),arr.shape[0],axis=0)
    #np.repeat(b_i.reshape(-1,1),arr.shape[1],axis=1)
    b_u_map=np.array(list(b_u)*arr.shape[0]).reshape(arr.shape)
    b_i_map=np.array(list(b_i)*arr.shape[1]).reshape(
        (arr.shape[1],arr.shape[0])).T
    
    #estimate b_i while fix b_u
    phi=arr-miu-b_u_map
    H_HT=np.array(
        [arr[i][~np.isnan(arr[i])].shape[0] for i in range(arr.shape[0])])
    numerator=[phi[i][~np.isnan(phi[i])].sum() for i in range(phi.shape[0])]
    b_i=np.divide(np.array(numerator),(lambda_+H_HT))
    
    #estimate b_u while fix b_i
    psi=arr-miu-b_i_map
    I_IT=np.array(
        [arr[:,u][~np.isnan(arr[:,u])].shape[0] for u in range(arr.shape[1])])
    numerator=[psi[:,u][~np.isnan(psi[:,u])].sum() for u in range(psi.shape[1])]
    b_u=np.divide(np.array(numerator),(lambda_+I_IT))
    
    #update repeat
    b_u_map=np.array(list(b_u)*arr.shape[0]).reshape(arr.shape)
    b_i_map=np.array(list(b_i)*arr.shape[1]).reshape(
        (arr.shape[1],arr.shape[0])).T
    
    #compute error
    epsilon=(arr-miu-b_u_map-b_i_map).ravel()
    error=np.square(epsilon[~np.isnan(epsilon)]).sum()
    
    return error,b_u,b_i

In [50]:
#als to estimate baseline model
def baseline_als(arr,miu_init=None,b_u_init=[],b_i_init=[],
             lambda_=0.02,tau=0.0001,
             max_iter=20,diagnosis=True
             ):

    #initialize
    stop=False
    counter=0
    sse=None
    
    #global mean
    if not miu_init:       
        miu=arr[~np.isnan(arr)].mean()
    else:
        miu=miu_init
        
    #user baseline
    if len(b_u_init)==0:
        b_u=np.zeros(arr.shape[1])
    else:
        b_u=b_u_init
    
    #item baseline
    if len(b_i_init)==0:
        b_i=np.zeros(arr.shape[0])
    else:
        b_i=b_i_init
       
    #als
    while not stop:
        
        error,b_u,b_i=als_epoch(arr,miu,b_u,b_i,
                                lambda_)

        counter+=1

        #maximum number of epoch
        if counter>=max_iter:
            stop=True
            if diagnosis:
                print('Not converged.',
                      'Consider increase number of iterations or tolerance')
                
        #use sum of squared error to determine if converged
        sse_prev=sse
        sse=error
        
        if sse_prev and abs(sse/sse_prev-1)<=tau:
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return b_u,b_i

###### Run

In [51]:
#als to estimate baseline
b_u,b_i=baseline_als(arr_train,
         lambda_=lagrange_multiplier,tau=tolerance,
         max_iter=max_num_of_epoch)

4 iterations to reach convergence



In [52]:
#compute global mean
miu=arr_train[~np.isnan(arr_train)].mean()

#matrix completion
output=miu+np.repeat(
            b_u.reshape(1,-1),
            arr_train.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),arr_train.shape[1],axis=1)

In [53]:
#use mse as benchmark for comparison
mse_baseline=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('Baseline Model Mean Squared Error:',round(mse_baseline,3))

Baseline Model Mean Squared Error: 0.94


##### Pearson Correlation with baseline

&nbsp;

Since we are using baseline in KNN, it's also worth a while to switch the distance metric from MSD to Pearson Correlation with Baseline. This metric is a variation of Pearson Correlation in statistics.

$$\rho_{uv} = \frac{cov(u,v)}{\sigma_u \cdot \sigma_v} = \frac{ \sum\limits_{i \in I_{uv}}
        (r_{ui} -  \mu_u) \cdot (r_{vi} - \mu_{v})} {\sqrt{\sum\limits_{i
        \in I_{uv}} (r_{ui} -  \mu_u)^2} \cdot \sqrt{\sum\limits_{i \in
        I_{uv}} (r_{vi} -  \mu_{v})^2} }$$
        
To replace $\mu_u$ and $\mu_{v}$ with $b_u$ and $b_{v}$ in the equation, we obtain the designated metric – Pearson Correlation with Baseline.

$$\text{similarity}(u, v) = \frac{
            \sum\limits_{i \in I_{uv}} (r_{ui} -  b_{u}) \cdot (r_{vi} -
            b_{v})} {\sqrt{\sum\limits_{i \in I_{uv}} (r_{ui} -  b_{u})^2}
            \cdot \sqrt{\sum\limits_{i \in I_{uv}} (r_{vi} -  b_{v})^2}}$$ 

###### Functions

In [54]:
#obtain pearson correlation with baseline
#if b_u is substituted with arr.mean(axis=1) and arr is a complete matrix
#the function will be equivalent to np.corrcoef(arr.T)
#except the diagonal line would be filled with zeros
@numba.njit
def get_pearson_corr_baseline_matrix(arr,b_u):

    similarity_matrix=np.zeros((arr.shape[1],arr.shape[1]))
    for u in range(arr.shape[1]):
        for v in range(u+1,arr.shape[1]):
            
            #self correlation distorts knn selection
            if u==v:
                continue

            #find items rated by both user u and v
            arr_sub1=arr[:,u]
            arr_sub2=arr[:,v]
            set_u=np.where(~np.isnan(arr_sub1))
            set_v=np.where(~np.isnan(arr_sub2))
            I_uv=set(set_u[0]).intersection(set(set_v[0]))
          
            #avoid the case where two users havent rated any items in common
            if len(I_uv)>0:
                
                #extract ratings of inner join items from user u and v
                arr_u=arr_sub1[np.array(list(I_uv))]
                arr_v=arr_sub2[np.array(list(I_uv))]
                numerator=(arr_u-b_u[u]).T@(arr_v-b_u[v])
                denominator=np.sqrt(
                    (arr_u-b_u[u]).T@(arr_u-b_u[u]))*np.sqrt(
                    (arr_v-b_u[v]).T@(arr_v-b_u[v]))
                pearson_baseline=numerator/denominator
            else:
                pearson_baseline=0

            #symmetric matrix
            similarity_matrix[u,v]=pearson_baseline
            similarity_matrix[v,u]=pearson_baseline
            
    return similarity_matrix

###### Run

In [55]:
#compute similarity matrix
similarity_matrix=get_pearson_corr_baseline_matrix(arr,b_u)

##### KNN with baseline

&nbsp;

There is nothing trivial about KNN with baseline. Like Pearson Correlation with baseline, we replace $\mu_u$ and $\mu_v$ with $b_u$ and $b_v$ in KNN with mean. Ça y est!

$$\hat{r}_{ui} = b_{u} + \frac{  \sum_{v \in N^k_i(u)} \mathcal{similarity}(u, v) \cdot (r_{vi}-b_{v})}  {\sum_{v \in N^k_i(u)} \mathcal{similarity}(u, v)}$$

&nbsp;

Reference to the paper with KNN with baseline (equation 3)

https://people.engr.tamu.edu/huangrh/Spring16/papers_course/matrix_factorization.pdf

###### Functions

In [56]:
#knn with baseline for matrix completion
@numba.njit
def knn_with_baseline(arr,similarity_matrix,
                      b_u,top_k=10):
    
    estimated=arr.copy()
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            if np.isnan(arr[i,u]):

                #find top k neighbor based upon similarity matrix
                rated_users=np.where(~np.isnan(arr[i]))[0]
                similarities=similarity_matrix[u][rated_users]
                top_k_neighbors=np.argsort(similarities)[-top_k:]
                N_k_i_u=np.array([
                    rated_users[neighbor] for neighbor in top_k_neighbors])

                #compute weighted average                
                if len(N_k_i_u)!=0:
                    numerator=(arr[i][
                        N_k_i_u]-b_u[
                        N_k_i_u]).T@similarity_matrix[u][N_k_i_u]
                    denominator=similarity_matrix[u][N_k_i_u].sum()
                    if denominator!=0:
                        estimated[i,u]=b_u[u]+numerator/denominator
                        continue
                        
                #when the users who rated item i
                #have nothing in common with user u
                #take the average of user baseline and item baseline
                #if both user baseline and item baseline are empty
                #take global baseline
                miu_i=arr[i][~np.isnan(arr[i])]
                miu_u=arr[:,u][~np.isnan(arr[:,u])]                
                if len(miu_i)==0 and len(miu_u)==0:
                    estimated[i,u]=arr[np.where(~np.isnan(arr))[0]][
                        np.where(~np.isnan(arr))[1]].mean()
                else:
                    estimated[i,u]=np.array(list(miu_i)+list(miu_u)).mean()
                    
    return estimated

###### Run

In [57]:
#matrix completion
output=knn_with_baseline(arr_train,similarity_matrix,b_u,top_k=top_k)

In [58]:
#use mse as benchmark for comparison
mse_knn_with_baseline=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('KNN with baseline Mean Squared Error:',round(mse_knn_with_baseline,3))

KNN with baseline Mean Squared Error: 0.85


#### Koren Neighborhood Model

&nbsp;

To facilitate global optimization, Koren proposed a neighborhood model with global weights independent of a specific user. The similarity weight is solved via optimization rather than correlation matrix. However, the model below is a bit different from the original version. As the only implicit feedback we get from the data is the items rated by the users, there is no point to create an extra variable which is merely a linear combination of neighbor-weighted deviation from the baseline. The official optimization problem is formulated as below. The notations are almost the same with everything above.

$$ \min_{w_*,b_*}\,\sum_{r_{ui}\,\in\,\mathcal{K}} \left(r_{ui} - \mu - b_u - b_i - |N_i^k(u)|^{-\frac{1}{2}} \sum_{j \in N_i^k(u)} (r_{uj} - \mu - b_u - b_j)w_{ij} \right)^2 + \lambda\left( \sum_{j \in N_i^k(u)}w_{ij}^2 + b_u^2 + b_i^2 \right)$$

where 

$w_{ij}$ denotes the similarity weight between item $i$ and item $j$

&nbsp;

The actual optimization solver is similar to Funk SVD with one more parameter $w_{ij}$. 

$$b_u := b_u + \alpha (\epsilon_{ui} - \lambda b_u)$$
$$b_i := b_i + \alpha (\epsilon_{ui} - \lambda b_i)$$
$$w_{ij} := w_{ij} + \alpha (|N(u)|^{-\frac{1}{2}} \cdot \epsilon_{ui} \cdot (r_{uj} - \mu - b_u - b_j) - \lambda w_{ij})$$

&nbsp;

Reference to the original paper of Koren neighborhood model (equation 11)

https://people.engr.tamu.edu/huangrh/Spring16/papers_course/matrix_factorization.pdf

###### Functions

In [59]:
#use numba to dramatically boost the speed of linear algebra
@numba.njit
def koren_neighbor_epoch(arr,miu,b_u,b_i,
                         similarity_matrix,
                         w_ij,alpha,lambda_,top_k):
    
    #initialize
    error=0
    
    #only iterate known ratings
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            r_ui=arr[i,u]
            
            #another way to identify nan
            #r_ui!=r_ui
            if np.isnan(r_ui):
                continue
                
            #find top k neighbor based upon similarity matrix
            rated_items=np.where(~np.isnan(arr[:,u]))[0]
            similarities=similarity_matrix[i][rated_items]
            top_k_neighbors=np.argsort(similarities)[-top_k:]
            N_k_i_u=np.array([
                    rated_items[neighbor] for neighbor in top_k_neighbors])
            
            #compute error
            if len(N_k_i_u)!=0:
                deviation=arr[:,u][N_k_i_u]-miu-b_u[u]-b_i[N_k_i_u]
                weighted_sum=(w_ij[i][N_k_i_u]).T@deviation
                epsilon_ui=(r_ui-miu-b_u[u]-b_i[i]-weighted_sum).item()
                error+=epsilon_ui**2
            else:
                epsilon_ui=(r_ui-miu-b_u[u]-b_i[i]).item()
                error+=epsilon_ui**2

            #update baseline
            b_u[u]+=alpha*(epsilon_ui-lambda_*b_u[u])
            b_i[i]+=alpha*(epsilon_ui-lambda_*b_i[i])
            
            #only update weights when there are similar items                
            if len(N_k_i_u)!=0:
                N_k_i_u_norm_sqrt=N_k_i_u.shape[0]**0.5                
                w_ij[i][N_k_i_u]=w_ij[i][N_k_i_u]+alpha*(
                    epsilon_ui*deviation/N_k_i_u_norm_sqrt-lambda_*w_ij[
                        i][N_k_i_u])
                w_ij[N_k_i_u][:,i]=w_ij[i][N_k_i_u]
                                        
    return error,b_u,b_i,w_ij

In [60]:
#koren weighted neighborhood model
def koren_neighbor(arr,similarity_matrix,
                   miu_init=None,b_u_init=[],
                   b_i_init=[],w_ij_init=[],
                   alpha=0.005,lambda_=0.02,
                   tau=0.0001,top_k=10,
                   max_iter=20,diagnosis=True):

    #initialize
    stop=False
    counter=0
    sse=None
    
    #global mean
    if not miu_init:       
        miu=arr[~np.isnan(arr)].mean()
    else:
        miu=miu_init
        
    #user baseline
    if len(b_u_init)==0:
        b_u=np.zeros(arr.shape[1])
    else:
        b_u=b_u_init
    
    #item baseline
    if len(b_i_init)==0:
        b_i=np.zeros(arr.shape[0])
    else:
        b_i=b_i_init
        
    #weighted neighbor
    if len(w_ij_init)==0:
        w_ij=np.zeros((arr.shape[0],arr.shape[0]))
        w_ij.fill(0.001)
    else:
        w_ij=w_ij_init
    
    #gradient descent
    while not stop:
        
        error,b_u,b_i,w_ij=koren_neighbor_epoch(
                         arr,miu,b_u,b_i,
                         similarity_matrix,
                         w_ij,alpha,lambda_,top_k)

        counter+=1
        
        #maximum number of epoch
        if counter>=max_iter:
            stop=True
            if diagnosis:
                print('Not converged.',
                      'Consider increase number of iterations or tolerance')
                
        #use sum of squared error to determine if converged
        sse_prev=sse
        sse=error
        if sse_prev and abs(sse/sse_prev-1)<=tau:
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return b_u,b_i,w_ij

###### Run

In [61]:
#compute item similarity matrix
similarity_matrix=get_msd_similarity_matrix(
    arr_train.T)

In [62]:
#koren neighbor
b_u,b_i,w_ij=koren_neighbor(arr_train,
                   similarity_matrix,
                   alpha=learning_rate,
                   lambda_=lagrange_multiplier,
                   tau=tolerance,top_k=top_k,
                   max_iter=max_num_of_epoch)

310 iterations to reach convergence



In [63]:
#compute deviation
deviation=arr_train-np.repeat(
            b_u.reshape(1,-1),
            arr_train.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),
    arr_train.shape[1],axis=1)-miu

#set nan to zero for dot product
deviation[np.isnan(deviation)]=0

#find top k neighbors for each item
top_k_neighbors=np.argsort(similarity_matrix,axis=0)[-top_k:]

#identify the col index of top k neighbors
col=top_k_neighbors.flatten()

#identify the row index of each item
row=np.array([j for i in range(arr_train.shape[0]) for j in [i]*top_k])

#create weights matrix
weights=np.zeros(w_ij.shape)

#only keep weights of top k neighbors
weights[(row,col)]=w_ij[(row,col)]

#compute influence from weighted neighbors
weighted_neighbors=weights@deviation

In [64]:
#matrix completion
output=miu+np.repeat(
            b_u.reshape(1,-1),
            arr_train.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),
    arr_train.shape[1],axis=1)+weighted_neighbors

In [65]:
#use mse as benchmark for comparison
mse_koren_ngbr=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('Koren Neighborhood Model Mean Squared Error:',round(mse_koren_ngbr,3))

Koren Neighborhood Model Mean Squared Error: 1.141


#### Koren Integrated Model

&nbsp;

There is nothing special about Koren Integrated Model. It is a combination of SVD++ and Koren Neighborhood Model which intends to get the best from both worlds. The official optimization problem is the aggregation of both.

$$ \min_{w_*,p_*,q_*,b_*,y_*}\,\sum_{r_{ui}\,\in\,\mathcal{K}} \left(r_{ui} - \mu - b_u - b_i - |N_i^k(u)|^{-\frac{1}{2}} \sum_{j \in N_i^k(u)} (r_{uj} - \mu - b_u - b_j)w_{ij} - q_i^T\left(p_u+|N(u)|^{-\frac{1}{2}} \sum_{j \in N(u)}y_j\right) \right)^2 + \lambda\left( ||p_u||^2 + ||q_i||^2 + \sum_{j \in N(u)}||y_j||^2 + \sum_{j \in N_i^k(u)}w_{ij}^2 + b_u^2 + b_i^2 \right)$$

&nbsp;

The actual optimization solver is the aggregation of both as well. 

$$b_u := b_u + \alpha (\epsilon_{ui} - \lambda b_u)$$
$$b_i := b_i + \alpha (\epsilon_{ui} - \lambda b_i)$$
$$w_{ij} := w_{ij} + \alpha (|N(u)|^{-\frac{1}{2}} \cdot \epsilon_{ui} \cdot (r_{uj} - \mu - b_u - b_j) - \lambda w_{ij})$$
$$p_u := p_u + \alpha (\epsilon_{ui} \cdot q_i - \lambda p_u)$$
$$q_i := q_i + \alpha (\epsilon_{ui} \cdot (p_u+|N(u)|^{-\frac{1}{2}} \sum_{j \in N(u)}y_j) - \lambda q_i)$$
$$y_j := y_j + \alpha (|N(u)|^{-\frac{1}{2}} \cdot \epsilon_{ui} \cdot q_i - \lambda y_j)$$

&nbsp;

Reference to the original paper of Koren integrated model (equation 16)

https://people.engr.tamu.edu/huangrh/Spring16/papers_course/matrix_factorization.pdf

###### Functions

In [66]:
#use numba to dramatically boost the speed of linear algebra
@numba.njit
def koren_integrated_epoch(arr,similarity_matrix,
                           miu,b_u,b_i,p_u,q_i,y_j,
                           w_ij,alpha,lambda_,top_k):
    
    #initialize
    error=0
    
    #only iterate known ratings
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            r_ui=arr[i,u]
            
            #another way to identify nan
            #r_ui!=r_ui
            if np.isnan(r_ui):
                continue
                
            #compute implicit feedback
            N_u=np.where(~np.isnan(arr[:,u]))[0]
            N_u_norm_sqrt=arr[:,u][~np.isnan(arr[:,u])].shape[0]**0.5
            feedback=(y_j[N_u,:]/N_u_norm_sqrt).sum(axis=0)

            #find top k neighbor based upon similarity matrix
            rated_items=np.where(~np.isnan(arr[:,u]))[0]
            similarities=similarity_matrix[i][rated_items]
            top_k_neighbors=np.argsort(similarities)[-top_k:]
            N_k_i_u=np.array([
                    rated_items[neighbor] for neighbor in top_k_neighbors])
            
            #compute error
            if len(N_k_i_u)!=0:
                deviation=arr[:,u][N_k_i_u]-miu-b_u[u]-b_i[N_k_i_u]
                weighted_sum=(w_ij[i][N_k_i_u]).T@deviation
                epsilon_ui=(r_ui-miu-b_u[u]-b_i[
                    i]-weighted_sum-q_i[i].T@(
                feedback+p_u[u]).reshape(-1,1)).item()
                error+=epsilon_ui**2
            else:
                epsilon_ui=(r_ui-miu-b_u[u]-b_i[
                    i]-q_i[i].T@(
                feedback+p_u[u]).reshape(-1,1)).item()
                error+=epsilon_ui**2

            #update
            b_u[u]+=alpha*(epsilon_ui-lambda_*b_u[u])
            b_i[i]+=alpha*(epsilon_ui-lambda_*b_i[i])
            p_u[u]+=alpha*(epsilon_ui*q_i[i]-lambda_*p_u[u])
            q_i[i]+=alpha*(epsilon_ui*(p_u[u]+feedback)-lambda_*q_i[i])
            y_j[N_u]+=alpha*(epsilon_ui*q_i[N_u]/N_u_norm_sqrt-lambda_*y_j[N_u])
            
            #only update weights when there are similar items                
            if len(N_k_i_u)!=0:
                N_k_i_u_norm_sqrt=N_k_i_u.shape[0]**0.5                
                w_ij[i][N_k_i_u]=w_ij[i][N_k_i_u]+alpha*(
                    epsilon_ui*deviation/N_k_i_u_norm_sqrt-lambda_*w_ij[
                        i][N_k_i_u])
                w_ij[N_k_i_u][:,i]=w_ij[i][N_k_i_u]
                  
    return error,b_u,b_i,p_u,q_i,y_j,w_ij

In [67]:
#koren integrated model
def koren_integrated(arr,similarity_matrix,
                     miu_init=None,b_u_init=[],
                     b_i_init=[],p_u_init=[],
                     q_i_init=[],y_j_init=[],
                     w_ij_init=[],
                     num_of_rank=40,alpha=0.005,
                     lambda_=0.02,tau=0.0001,top_k=10,
                     max_iter=20,diagnosis=True
                     ):

    #initialize
    stop=False
    counter=0
    sse=None
    
    #global mean
    if not miu_init:       
        miu=arr[~np.isnan(arr)].mean()
    else:
        miu=miu_init
        
    #user baseline
    if len(b_u_init)==0:
        b_u=np.zeros(arr.shape[1])
    else:
        b_u=b_u_init
    
    #item baseline
    if len(b_i_init)==0:
        b_i=np.zeros(arr.shape[0])
    else:
        b_i=b_i_init
        
    #user latent factors
    if len(p_u_init)==0:
        p_u=np.zeros((arr.shape[1],num_of_rank))
        p_u.fill(0.1)
    else:
        p_u=p_u_init
    
    #item latent factors
    if len(q_i_init)==0:
        q_i=np.zeros((arr.shape[0],num_of_rank))
        q_i.fill(0.1)
    else:
        q_i=q_i_init
        
    #user implicit feedback
    if len(y_j_init)==0:
        y_j=np.zeros((arr.shape[0],num_of_rank))
        y_j.fill(0.1)
    else:
        y_j=y_j_init
        
    #weighted neighbor
    if len(w_ij_init)==0:
        w_ij=np.zeros((arr.shape[0],arr.shape[0]))
        w_ij.fill(0.001)
    else:
        w_ij=w_ij_init
    
    #gradient descent
    while not stop:
        
        error,b_u,b_i,p_u,q_i,y_j,w_ij=koren_integrated_epoch(
                           arr,similarity_matrix,
                           miu,b_u,b_i,p_u,q_i,y_j,
                           w_ij,alpha,lambda_,top_k)

        counter+=1
        
        #maximum number of epoch
        if counter>=max_iter:
            stop=True
            if diagnosis:
                print('Not converged.',
                      'Consider increase number of iterations or tolerance')
                
        #use sum of squared error to determine if converged
        sse_prev=sse
        sse=error
        if sse_prev and abs(sse/sse_prev-1)<=tau:
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return b_u,b_i,p_u,q_i,y_j,w_ij

###### Run

In [68]:
b_u,b_i,p_u,q_i,y_j,w_ij=koren_integrated(
                 arr_train,similarity_matrix,
                 num_of_rank=num_of_latent_factors,
                 alpha=learning_rate,
                 lambda_=lagrange_multiplier,tau=tolerance,
                 max_iter=max_num_of_epoch)

292 iterations to reach convergence



In [69]:
#compute implicit feedback
feedback=[]
for u in range(arr_train.shape[1]):
    N_u=np.where(~np.isnan(arr_train[:,u]))[0]
    N_u_norm_sqrt=arr_train[:,u][~np.isnan(arr_train[:,u])].shape[0]**0.5
    feedback.append((y_j[N_u,:]/N_u_norm_sqrt).sum(axis=0))

#compute deviation
deviation=arr_train-np.repeat(
            b_u.reshape(1,-1),
            arr_train.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),
    arr_train.shape[1],axis=1)-miu

#set nan to zero for dot product
deviation[np.isnan(deviation)]=0

#find top k neighbors for each item
top_k_neighbors=np.argsort(similarity_matrix,axis=0)[-top_k:]

#identify the col index of top k neighbors
col=top_k_neighbors.flatten()

#identify the row index of each item
row=np.array([j for i in range(arr_train.shape[0]) for j in [i]*top_k])

#create weights matrix
weights=np.zeros(w_ij.shape)

#only keep weights of top k neighbors
weights[(row,col)]=w_ij[(row,col)]

#compute influence from weighted neighbors
weighted_neighbors=weights@deviation

In [70]:
#matrix completion
output=miu+np.repeat(
            b_u.reshape(1,-1),
            arr_train.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),
    arr_train.shape[1],axis=1)+q_i@(
    p_u+np.mat(feedback)).T+weighted_neighbors

In [71]:
#use mse as benchmark for comparison
mse_koren_integrated=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('Koren Integrated Model Mean Squared Error:',
      round(mse_koren_integrated,3))

Koren Integrated Model Mean Squared Error: 1.143
